In [1]:
import transformers
from transformers import DistilBertTokenizer,BertTokenizer
import pandas as pd
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification,TFTrainingArguments
from transformers import TFTrainer

c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rvaib\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rvaib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rvaib\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
train_df = pd.read_csv('data/train.csv')
# test_x =pd.read_csv('data/test.csv') 
# test_y =pd.read_csv('data/test_labels.csv')


In [4]:
# test_df = pd.merge(test_x,test_y,how='inner',on='id')
# test_df =test_df[test_df["toxic"]!=-1].reset_index(drop=True)
# test_df.head()

In [5]:
train_df =train_df.sample(n=10000,random_state=42)
# test_df =test_df.sample(n=2000,random_state=42)


In [6]:
stop_words =set(stopwords.words('english'))
lemmatizer =WordNetLemmatizer()

In [7]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens ]
    return ' '.join(tokens)

In [8]:
train_df['cleaned_text'] = train_df['comment_text'].apply(preprocess_text)



In [9]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_text
119105,7ca72b5b9c688e9e,"Geez, are you forgetful! We've already discus...",0,0,0,0,0,0,"geez , forgetful ! 've already discussed marx ..."
131631,c03f72fd8f8bf54f,Carioca RFA \n\nThanks for your support on my ...,0,0,0,0,0,0,carioca rfa thanks support request adminship ....
125326,9e5b8e8fc1ff2e84,"""\n\n Birthday \n\nNo worries, It's what I do ...",0,0,0,0,0,0,"`` birthday worry , 's ; ) enjoy ur day|talk|e ``"
111256,5332799e706665a6,Pseudoscience category? \n\nI'm assuming that ...,0,0,0,0,0,0,pseudoscience category ? 'm assuming article p...
83590,dfa7d8f0b4366680,"(and if such phrase exists, it would be provid...",0,0,0,0,0,0,"( phrase exists , would provided search engine..."
...,...,...,...,...,...,...,...,...,...
25725,4412b93f983379a4,"Sam, get real! No-one's gunner believe all thi...",0,0,0,0,0,0,"sam , get real ! no-one 's gunner believe stuf..."
46449,7c1bd75a71704cc2,Get a life \n\nwhy dont you go get a life and ...,1,0,0,0,1,0,get life dont go get life stop editing article...
108844,45eed1057ea7a582,"Nothing a sock puppet(and trust me, I know wha...",0,0,0,0,0,0,"nothing sock puppet ( trust , know mean , sure..."
142353,f9718cf5cbdd9a25,"""\nThe lead section is a summary of the main p...",0,0,0,0,0,0,"`` lead section summary main point , mention i..."


In [10]:
X =list(train_df['cleaned_text'])
y = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y, test_size=0.20,random_state=42)

In [12]:
X_train,y_train

(['16:20 , 24 october 2008',
  'message shadeed hi , n ! gger !',
  "`` know charged shooting u personnel , convicted , hedged description instead saying shot u personnel , said indicted shot . added convicted , since know convicted able report fact shot , without attenuating `` '' according indictment ( conviction ) '' '' language , ? ``",
  'february 2008 please stop adding inappropriate external link wikipedia . considered spamming wikipedia vehicle advertising promotion . since wikipedia us nofollow tag , addition link wikipedia alter search engine ranking . continue spamming , blocked editing wikipedia .',
  "magazine exist ? n't allowed publish magazine something harmful brain also illegal . think society catering stoner little much .",
  "`` reading preface `` '' making rumour '' '' page xiv mr. caillat say ... '' '' came together create rumour record plant sausalito '' '' , 294 talk hot hollywood , look like record plant used ? ``",
  "`` # `` '' '' talk ''",
  'welcome ! hello

In [13]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encoding =tokenizer(X_train,truncation=True,padding=True)
test_encoding =tokenizer(X_test,truncation=True,padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
  dict(train_encoding),y_train  
))


In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encoding),y_test
))

In [ ]:
training_args =TFTrainingArguments(
    output_dir='C:/Users/rvaib/OneDrive/Desktop/nlp_with_NLTK/nlp/content/output1',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content',
    logging_steps=10,
    eval_steps=1
)

In [ ]:
with training_args.strategy.scope():
    model =TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
trainer =TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)  

In [ ]:
trainer.train()
